# 第2章 プロンプト作成の原則

本章の主要な内容はプロンプト作成の原則についてです。本章では、プロンプト作成における2つの原則といくつかの関連戦略を説明し、効果的なプロンプトを作成する練習ができます。これにより、LLMを便利で効果的に使用できるようになります。

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一環境設定" data-toc-modified-id="一、環境設定">一、環境設定</a></span></li>
        <li>
            <span><a href="#二つの基本原則" data-toc-modified-id="二、二つの基本原則">二、二つの基本原則</a></span>
            <ul class="toc-item">
                <li><span><a href="#21-原則一明確で具体的な指示を書く" data-toc-modified-id="2.1 原則一：明確で具体的な指示を書く">2.1 原則一：明確で具体的な指示を書く</a></span></li>
                <li><span><a href="#22-原則二モデルに考える時間を与える" data-toc-modified-id="2.2 原則二：モデルに考える時間を与える">2.2 原則二：モデルに考える時間を与える</a></span></li>
            </ul>
        </li>
        <li><span><a href="#三制限事項" data-toc-modified-id="三、制限事項">三、制限事項</a></span>
        </li>
    </ul>
</div>

## 一、環境設定

本チュートリアルはOpenAIが公開しているChatGPT APIを使用しています。そのため、まずChatGPTのAPI_KEYを取得する必要があります（または公式サイトに直接アクセスしてオンラインテストも可能です）。次に、OpenAIのサードパーティライブラリをインストールしてください。簡便性と互換性を考慮して、本チュートリアルでは```Python 3```環境における```openai.api_key```メソッドベースの設定方法を紹介します。環境変数ベースの設定方法もあります。詳細については[OpenAI公式ドキュメント](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)をご参照ください。

まず、OpenAIライブラリをインストールする必要があります：
```bash
pip install openai
```

In [ ]:
import openai
# サードパーティライブラリのインポート

openai.api_key  = "sk-..."
# API_KEYの設定、ご自身のAPI_KEYに置き換えてください

# 以下は環境変数ベースの設定方法の例で、より安全です。参考程度で、以降では扱いません。
# import openai
# import os
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

このコース全体ではgpt-3.5-turboモデルを例として使用します。後のコースでOpenAIが提供する[Chat Completions API](https://platform.openai.com/docs/guides/gpt/chat-completions-api)の使用方法について詳しく学習します。ここでは、それを関数として包装します。内部の仕組みを知る必要はありません。この関数を呼び出し、プロンプトを入力パラメータとして渡すと、対応するCompletion（回答結果）が出力されることを理解していただければ十分です。

In [ ]:
# OpenAI APIをラップする関数、パラメータはプロンプト、対応する結果を返す
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 対応するプロンプトテキスト
    model: 呼び出すモデル、デフォルトはgpt-3.5-turbo(ChatGPT)、ベータ版資格を持つユーザーはgpt-4を選択可能
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # モデル出力の温度係数、出力のランダム性を制御
    )
    # OpenAIのChatCompletion APIを呼び出し
    return response.choices[0].message["content"]

## 二、二つの基本原則

### 2.1 原則一：明確で具体的な指示を書く

できるだけ明確で具体的な指示を提供することで、モデルに実行してもらいたい操作を表現する必要があります。これにより、モデルが正しい出力を生成するように導き、無関係または不正確な応答を得る可能性を減らします。明確な指示は簡潔である必要があるという意味ではありません。多くの場合、より長いプロンプトの方が実際により明確で、より多くのコンテキストを提供し、より詳細で関連性の高い出力を生成する可能性があります。

**2.1.1 区切り文字を使用して入力の異なる部分を明確に表す**

区切り文字には：```、""、<>、:、\<tag> \</tag>などがあります。

明らかな句読点を使用して、特定のテキスト部分をプロンプトの残りの部分から分離できます。マークの形式に制限はなく、モデルがこれが独立した部分であることを明確に知ることができればよいのです。区切り文字の使用は、プロンプトインジェクション（Prompt injection）を効果的に回避できます。プロンプトインジェクションとは、ユーザーが（開発者が事前定義した）プロンプトに何らかの入力を追加することが許可されている場合、提供された指示が開発者が実行したい操作と競合する可能性があり、それによってLLMがユーザー入力の指示に従い、開発者が期待する操作を実行しなくなることです。つまり、入力には他の指示が含まれている可能性があり、これがあなたの指示を上書きしてしまいます。これに対して、区切り文字の使用は良い戦略です。

以下の例では、テキストを提供してGPTに要約を求めています。この例では、```を区切り文字として使用しています。

In [ ]:
# 中文版见下一个 cell
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.

In [ ]:
text = f"""
できるだけ明確で具体的な指示を提供して、モデルに実行してもらいたいタスクを表現する必要があります。\
これにより、モデルが必要な出力に向かって導かれ、無関係または不正確な応答を受け取る可能性が低くなります。\
明確なプロンプトを書くことと簡潔なプロンプトを書くことを混同してはいけません。\
多くの場合、より長いプロンプトはモデルにより多くの明確性とコンテキスト情報を提供し、より詳細で関連性の高い出力につながります。
"""
# 要約するテキスト内容
prompt = f"""
三つのバッククォートで囲まれたテキストを一文で要約してください。
```{text}```
"""
# 指示内容、```を使用して指示と要約対象のコンテンツを分離
response = get_completion(prompt)
print(response)

明確で具体的な指示を提供し、無関係または不正確な応答を避け、明確さと簡潔さを混同せず、より長いプロンプトはより多くの明確性とコンテキスト情報を提供し、より詳細で関連性の高い出力につながります。

**2.1.2 構造化された出力を求める**

出力はJson、HTMLなどの形式にすることができます。

第二の戦略は、構造化された出力を生成するよう求めることです。これにより、モデルの出力をより解析しやすくなります。例えば、Pythonで辞書やリストに読み込むことができます。

以下の例では、3冊の本のタイトル、著者、カテゴリを生成するようGPTに求め、JSON形式で返すよう要求しています。解析を容易にするため、JSONのキーを指定しています。

In [7]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)


[
  {
    "book_id": 1,
    "title": "The Lost City of Zorath",
    "author": "Aria Blackwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Survivors",
    "author": "Ethan Stone",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Secret Life of Bees",
    "author": "Lila Rose",
    "genre": "Romance"
  }
]


In [ ]:
prompt = f"""
書名、著者、ジャンルを含む3冊の架空の書籍リストを生成し、\
次のキーを含むJSON形式で提供してください：book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)

**2.1.3 モデルに条件を満たしているかどうかをチェックするよう求める**

タスクが必ずしも満たされるとは限らない前提（条件）を含む場合、まずこれらの前提をチェックするようモデルに伝え、満たされていない場合は指摘して後続の完全なフローの実行を停止することができます。また、起こりうるエッジケースとモデルの対応も考慮して、予期しない結果やエラーの発生を回避することもできます。

以下の例では、2つのテキストをモデルに別々に提供します。1つは茶の入れ方の手順、もう1つは明確な手順のないテキストです。モデルに、一連の指示が含まれているかどうかを判断し、含まれている場合は指定された形式で指示を書き直し、含まれていない場合は「手順が提供されていません」と回答するよう求めます。

In [9]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!




In [11]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:
Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


In [ ]:
# 条件を満たす入力（textに手順が提供されている）
text_1 = f"""
お茶を淹れるのは簡単です。まず、お湯を沸かします。\
待っている間に、カップを取り茶袋を入れます。\
お湯が十分熱くなったら、茶袋にお湯を注ぎます。\
しばらく待って、茶葉を蒸らします。数分後、茶袋を取り出します。\
お好みで、砂糖やミルクを加えて味を調えることができます。\
これで、美味しいお茶をお楽しみいただけます。
"""
prompt = f"""
三つの引用符で囲まれたテキストが提供されます。\
一連の指示が含まれている場合は、次の形式でその指示を書き直してください：

第1ステップ - ...
第2ステップ - …
…
第Nステップ - …

テキストに一連の指示が含まれていない場合は、「手順が提供されていません」と直接書いてください。
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1の要約:")
print(response)

In [ ]:
# 条件を満たさない入力（textに期待される指示が提供されていない）
text_2 = f"""
今日は太陽が明るく輝き、鳥たちが歌っています。\
公園を散歩するのに美しい日です。\
花が咲き誇り、木々が微風に優しく揺れています。\
人々は外に出てこの美しい天気を楽しんでおり、ピクニックをしている人もいれば、ゲームをしたり草地でリラックスしたりしている人もいます。\
屋外で過ごし、自然の美しさを鑑賞するのに完璧な日です。
"""
prompt = f"""
三つの引用符で囲まれたテキストが提供されます。\
一連の指示が含まれている場合は、次の形式でその指示を書き直してください：

第1ステップ - ...
第2ステップ - …
…
第Nステップ - …

テキストに一連の指示が含まれていない場合は、「手順が提供されていません」と直接書いてください。
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2の要約:")
print(response)

**2.1.4 少数の例を提供する**（少数例プロンプト、Few-shot prompting）

つまり、モデルに実際のタスクを実行させる前に、タスクを成功裏に実行した少数の例を提供することです。

例えば、以下の例では、モデルのタスクが一貫したスタイルで質問に回答することであることを伝え、まず子供と祖父の間の対話の例を提供します。子供が「忍耐について教えて」と言い、祖父が以下のようなスタイルの比喩で回答します。モデルに一貫したトーンで回答するよう既に伝えているため、今度は「レジリエンスについて教えて」と質問すると、モデルがこの少数例（few-shot example）を持っているため、似たようなトーンで次のタスクに回答します。

In [13]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends with the wind but never breaks. It is the ability to bounce back from adversity and keep moving forward, even when things get tough. Just like a tree that grows stronger with each storm it weathers, resilience is a quality that can be developed and strengthened over time.


In [ ]:
prompt = f"""
あなたのタスクは一貫したスタイルで質問に回答することです。

<子供>: 忍耐について教えて。

<祖父母>: 最も深い峡谷を掘る川は目立たない泉から流れ出し、最も壮大な交響曲は単一の音符から始まり、最も複雑なタペストリーは一本の孤独な糸から編み始められます。

<子供>: レジリエンス（復元力）について教えて。
"""
response = get_completion(prompt)
print(response)

### 2.2 モデルに考える時間を与える

モデルの推論プロセスが急ぎすぎて間違った結論に達していることが分かった場合、プロンプトを再構成して、最終的な回答を提供する前に**思考チェーン**を展開するか、一連の関連する推論（a chain or series of relevant reasoning）を行うようモデルに求める必要があります。つまり、短時間で、または少ない文字数で完了できない複雑なタスクをモデルに与えると、その出力結果にエラーが生じやすくなります。この状況は人間にとっても同様です：複雑な数学問題を解くのに十分な時間を与えずに誰かに求めると、その人もミスを犯す可能性があります。したがって、このような場合は、モデルに問題についてより多くの時間を考えるよう指示し、タスクにより多くの計算資源を費やすようにする必要があります。

**2.2.1 タスクを完了するために必要な手順を指定する**

次に、複雑なタスクを与え、そのタスクを完了するための一連の手順を示すことで、この戦略の効果を示します。

まず、ジャックとジルの物語を説明し、プロンプトに次の操作を実行するよう指示します：まず、三つのバッククォートで区切られたテキストを一文で要約する。第二に、要約をフランス語に翻訳する。第三に、フランス語の要約で各名前をリストアップする。第四に、次のキーを含むJSONオブジェクトを出力する：フランス語要約と名前の数。出力は改行文字で区切る。

In [16]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Two siblings, Jack and Jill, go on a quest to fetch water from a well on a hilltop, but misfortune strikes and they both tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed.

Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits sur une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts. 
Noms: Jack, Jill.

{
  "french_summary": "Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits sur une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.",
  "num_names": 2
}


In [ ]:
text = f"""
魅力的な村で、兄妹のジャックとジルが山頂の井戸に水を汲みに出発しました。\
楽しい歌を歌いながら登っていましたが、\
不運が襲いかかりました―ジャックが石につまずいて山から転がり落ち、ジルがその後に続きました。\
少々傷を負いましたが、彼らは温かい家に帰りつきました。\
このような事故にもかかわらず、彼らの冒険精神は衰えることなく、喜びに満ちて探索を続けています。
"""
# example 1
prompt_1 = f"""
次の操作を実行してください：
1-三つのバッククォートで囲まれた以下のテキストを一文で要約してください。
2-要約を英語に翻訳してください。
3-英語の要約で各人名をリストアップしてください。
4-次のキーを含むJSONオブジェクトを出力してください：English_summary、num_names。

改行文字で回答を区切ってください。

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("prompt 1:")
print(response)

上記の出力にはまだ一定の問題があります。例えば、キー「名前」がフランス語に置き換えられることがあります（翻訳注：英語原版では、対応する指示の第3ステップの出力が'Noms:'で、これはNameのフランス語です。この動作は予測困難で、エクスポートに困難をもたらす可能性があります）

そのため、プロンプトを改善します。このプロンプトの前半部分は変わらず、同時に**出力形式を正確に指定**します。

In [18]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in French summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: 兄妹杰克和吉尔在山顶井里打水时发生意外，但他们仍然保持冒险精神继续探索。
Translation: Jack and Jill, deux frères et sœurs, ont eu un accident en allant chercher de l'eau dans un puits de montagne, mais ils ont continué à explorer avec un esprit d'aventure.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack and Jill, deux frères et sœurs, ont eu un accident en allant chercher de l'eau dans un puits de montagne, mais ils ont continué à explorer avec un esprit d'aventure.", "num_names": 2}


In [ ]:
prompt_2 = f"""
1-<>で囲まれた以下のテキストを一文で要約してください。
2-要約を英語に翻訳してください。
3-英語の要約で各名前をリストアップしてください。
4-次のキーを含むJSONオブジェクトを出力してください：English_summary、num_names。

次の形式を使用してください：
テキスト：<要約するテキスト>
要約：<要約>
翻訳：<要約の翻訳>
名前：<英語要約の名前リスト>
JSONの出力：<English_summaryとnum_namesを含むJSON>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)

**2.2.2 結論を出す前に独自の解法を見つけるようモデルに指導する**

急いで決定を下す前に、自分で解決方法を考え出すようモデルに明確に指導します。時にはこれによってより良い結果が得られます。これは前述の思想と似ており、つまりモデルに考える時間を与えることです。

次に、問題と学生からの回答を提供し、回答が正しいかどうかをモデルに判断してもらいます：

In [22]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


In [ ]:
prompt = f"""
学生の解決方法が正しいかどうかを判断してください。

問題:
太陽光発電所を建設しており、財務計算の支援が必要です。

    土地費用は100ドル/平方フィート
    太陽光パネルを250ドル/平方フィートで購入可能
    メンテナンス契約を交渉済みで、年間固定10万ドル、さらに平方フィートあたり10ドル
    平方フィート数の関数として、初年度運営の総費用はいくらですか？

学生の解決方法：
xを発電所のサイズ（平方フィート単位）とする。
費用：

    土地費用：100x
    太陽光パネル費用：250x
    メンテナンス費用：100,000ドル+100x
    総費用：100x+250x+100,000ドル+100x=450x+100,000ドル
"""
response = get_completion(prompt)
print(response)

しかし注意してください、学生の解決方法は実際には間違っています。（*メンテナンス費用項目の100xは10xであるべきで、総費用450xは360xであるべきです*）

この問題は、モデルに最初に独自の解法を見つけるよう指導することで解決できます。

次のプロンプトでは、モデルに最初にこの問題を自分で解くよう求め、その後自分の解法と学生の解法を比較して、学生の解法が正しいかどうかを判断するよう求めます。同時に、出力形式の要求も与えています。タスクを分解し、手順を明確にすることで、モデルにより多くの考える時間を与え、時にはより正確な結果を得ることができます。この例では、学生の答えは間違っていますが、モデルに最初に自分で計算させなければ、学生が正しいと誤解される可能性があります。

In [24]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x

Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect


In [ ]:
prompt = f"""
学生の解決方法が正しいかどうかを判断してください。次の手順で問題を解決してください：

手順：
まず、自分で問題を解決し、問題を解く際に数式を列記してください。
その後、あなたの解決方法と学生の解決方法を比較し、学生の解決方法が正しいかどうかを評価してください。
自分で問題を完了する前に、学生の解決方法が正しいかどうかを決定しないでください。

次の形式を使用してください：

問題：問題文
学生の解決方法：学生の解決方法文
実際の解決方法と手順：実際の解決方法と手順文
**学生の計算結果：学生の計算結果文
実際の計算結果：実際の計算結果文
学生の計算結果と実際の計算結果は同じか：はい または いいえ
学生の解決方法と実際の解決方法は同じか：はい または いいえ**
学生の成績：正しい または 不正確

問題：
太陽光発電所を建設しており、財務計算の支援が必要です。
- 土地費用は平方フィートあたり100ドル
- 太陽光パネルを平方フィートあたり250ドルで購入可能
- メンテナンス契約を交渉済みで、年間固定10万ドル、さらに平方フィートあたり10ドル
平方フィート数の関数として、初年度運営の総費用はいくらですか？

学生の解決方法：

xを発電所のサイズ（平方フィート単位）とする。
費用：
1. 土地費用：100x
2. 太陽光パネル費用：250x
3. メンテナンス費用：100,000+100x
総費用：100x+250x+100,000+100x=450x+100,000

実際の解決方法と手順：
"""
response = get_completion(prompt)
print(response)

## 三、制限事項

**大規模モデル関連アプリケーションを開発する際は必ず心に留めておいてください：**

**虚偽の知識**：モデルは時に真実らしく見えるが実際には作り話の知識を生成することがある

モデルは訓練過程で大量の知識に触れましたが、見た情報を*完全に*記憶しているわけではないため、自分の知識の境界を正確に把握していません。これは、モデルが理解困難で曖昧なトピックについて質問に答えようとし、合理的に聞こえるが実際には正確でない答えを作り出す可能性があることを意味します。私たちはこれらの作り出された考えを幻覚（Hallucination）と呼びます。

以下の例は大規模モデルの幻覚を示しています。私たちはBoie社が生産する*AeroGlide UltraSlim Smart Toothbrush*製品の情報を教えるよう求めました。実際、この会社は実在しますが、製品は作り話で、モデルは真面目に自分が作り出した知識を提供し、しかも非常に紛らわしいものでした。

In [28]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that uses advanced sonic technology to provide a deep and thorough clean. It features a slim and sleek design that makes it easy to hold and maneuver, and it comes with a range of smart features that help you optimize your brushing routine.

One of the key features of the AeroGlide UltraSlim Smart Toothbrush is its advanced sonic technology, which uses high-frequency vibrations to break up plaque and bacteria on your teeth and gums. This technology is highly effective at removing even the toughest stains and buildup, leaving your teeth feeling clean and fresh.

In addition to its sonic technology, the AeroGlide UltraSlim Smart Toothbrush also comes with a range of smart features that help you optimize your brushing routine. These include a built-in timer that ensures you brush for the recommended two minutes, as well as a pressure sensor that alerts you if you're brushing too hard.

Overall, the AeroGlide UltraS

In [ ]:
prompt = f"""
Boie社が生産するAeroGlide UltraSlim Smart Toothbrushの関連情報を教えてください
"""
response = get_completion(prompt)
print(response)

虚偽と本物を見分けるのが非常に困難であるため、読者はこのシリーズのチュートリアルで学んだ知識に基づいて、自分のアプリケーションを構築する際に幻覚状況をできるだけ回避してください。幻覚は大規模モデルの既知の欠陥です（注：2023年7月時点）、OpenAIもこの問題の解決に努めています。

モデルにテキストに基づいて回答を生成してもらいたい場合、幻覚を減らすもう一つの戦略は、まずモデルにそのテキストから関連する引用情報（any relevant quotes）をすべて取得するよう求め、その後引用した情報に基づいて質問に答えるよう求めることです。これにより、回答をソース文書にたどることができ、通常は幻覚を減らすのに非常に役立ちます。

**バックスラッシュ使用に関する説明：**

本チュートリアルでは、読みやすさを向上させるためにテキストを画面サイズに合わせるためにバックスラッシュ\を使用し、改行文字\nは使用していません。GPT-3は改行文字（newline characters）の影響を受けませんが、他の大規模モデルを呼び出す際は、改行文字がモデルのパフォーマンスに影響するかどうかを追加で考慮する必要があります。